<a href="https://colab.research.google.com/github/fazal735/DL_A2/blob/main/DL_A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import torch
import torch.nn as nn


In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
train_file='/content/drive/MyDrive/inaturalist_12K/train'
test_file='/content/drive/MyDrive/inaturalist_12K/val'

NameError: name 'Aves' is not defined

In [10]:
!git clone https://github.com/fazal735/DL_A2

Cloning into 'DL_A2'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3/3), done.


In [11]:
# model definetion
class model(nn.Module):
  def __init__(self,num_filters=[32,64,128,256,512],filter_size=[3,3,5,5,7],activation=nn.ReLU(),
               stride=1, padding=1, pool_size=(2,2),fc_size=512,nom_o_classes=10,
               dropout=0,inchannels=3):
    super(model,self).__init__()
    self.num_filters=num_filters
    self.filter_size=filter_size
    self.activation=activation
    self.stride=stride
    self.padding=padding
    self.pool_size=pool_size
    self.fc_size=fc_size
    self.nom_o_classes=nom_o_classes
    self.dropout=dropout
    self.channels=inchannels


    def image_size(img_w,filter_size,padding,stride):
      return ((img_w-filter_size+2*padding)/stride+1)*0.5


    #layers of convolution
    #layer1
    self.conv_layer1=nn.Conv2d(self.channels,self.num_filters[0], stride=self.stride, padding=self.padding,
                               kernel_size=self.filter_size[0])
    self.dropout1=nn.Dropout2d(self.dropout)
    self.batch_norm1=nn.BatchNorm2d(self.num_filters[0])
    img_size1=image_size(224,self.filter_size[0],self.padding,self.stride)

    #layer2
    self.conv_layer2=nn.Conv2d(self.channels,self.num_filters[1], stride=self.stride, padding=self.padding,
                              kernel_size=self.filter_size[1])
    self.dropout2=nn.Dropout2d(self.dropout)
    self.batch_norm2=nn.BatchNorm2d(self.num_filters[1])
    img_size2=image_size(img_size1,self.filter_size[1],self.padding,self.stride)

    #layer3
    self.conv_layer3=nn.Conv2d(self.channels,self.num_filters[2], stride=self.stride, padding=self.padding,
                              kernel_size=self.filter_size[2])
    self.dropout3=nn.Dropout2d(self.dropout)
    self.batch_norm3=nn.BatchNorm2d(self.num_filters[2])
    img_size3=image_size(img_size2,self.filter_size[2],self.padding,self.stride)

    #layer4
    self.conv_layer4=nn.Conv2d(self.channels,self.num_filters[3], stride=self.stride, padding=self.padding,
                              kernel_size=self.filter_size[3])
    self.dropout4=nn.Dropout2d(self.dropout)
    self.batch_norm4=nn.BatchNorm2d(self.num_filters[3])
    img_size4=image_size(img_size3,self.filter_size[3],self.padding,self.stride)

    #layer5
    self.conv_layer5=nn.Conv2d(self.channels,self.num_filters[4], stride=self.stride, padding=self.padding,
                              kernel_size=self.filter_size[4])
    self.dropout5=nn.Dropout2d(self.dropout)
    self.batch_norm5=nn.BatchNorm2d(self.num_filters[4])
    img_size5=int(image_size(img_size4,self.filter_size[4],self.padding,self.stride))


    self.pool=nn.MaxPool2d(self.pool_size,stride=2)



    # forward
    def forward(self,x):
      #layer1
      x=self.conv_layer0(x)
      x=self.activation(x)
      x=self.pool(x)

      #layer2
      x=self.conv_layer1(x)
      x=self.activation(x)
      x=self.pool(x)

      #layer3
      x=self.conv_layer2(x)
      x=self.activation(x)
      x=self.pool(x)

      #layer4
      x=self.conv_layer3(x)
      x=self.activation(x)
      x=self.pool(x)

      #layer5
      x=self.conv_layer4(x)
      x=self.activation(x)
      x=self.pool(x)

      self.classifier=nn.Sequential(nn.Flatten(),nn.Linear('''tensor size'''),nn.ReLU(),
                                    nn.Linear(),nn.ReLU(),
                                    nn.Linear()
                                    )



model=model()
print(model)

model(
  (activation): ReLU()
  (conv_layer1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (dropout1): Dropout2d(p=0, inplace=False)
  (batch_norm1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_layer2): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (dropout2): Dropout2d(p=0, inplace=False)
  (batch_norm2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_layer3): Conv2d(3, 128, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (dropout3): Dropout2d(p=0, inplace=False)
  (batch_norm3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_layer4): Conv2d(3, 256, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (dropout4): Dropout2d(p=0, inplace=False)
  (batch_norm4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_layer5): Conv2d(3, 512, kernel_size=(7, 7), stride=(1, 1), padding

In [12]:
epochs=100
learning_rate=0.1

In [13]:
loss=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model.parameters(),lr=learning_rate)

In [ ]:
#training loop

for epoch in range(epochs):

  #fo

    #forwards pass

    #loss calculation

    # back porp

    #update grads

SyntaxError: incomplete input (<ipython-input-6-b1da103691c9>, line 13)

In [ ]:
class CNN_model(nn.Module):
  def __init__(self, input_features, num_filters=[32,64,128,256,512],filter_size=[3,3,5,5,7],activation=nn.ReLU(),
               stride=1, padding=1, pool_size=(2,2),fc_size=512,nom_o_classes=10,
               dropout=0,inchannels=3):
    super().__init__()

    self.features=nn.Sequential(
        nn.Conv2d(input_features,in_channels=3, num_filters[0], kernel_size=filter_size[0],stride=stride,padding=padding),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )

    self.classifier=nn.Sequential()

  def forward(self,x):
    x=self.features(x)
    x=self.classifier(x)

    return x

SyntaxError: positional argument follows keyword argument (<ipython-input-14-6300cac1d05e>, line 8)

In [25]:
import os
import pandas as pd
from torchvision import datasets,transforms

data=datasets.ImageFolder(root='/content/drive/MyDrive/inaturalist_12K/train/Amphibia',transform=transforms.ToTensor())


FileNotFoundError: Couldn't find any class folder in /content/drive/MyDrive/inaturalist_12K/train/Amphibia.